In [2]:
!pip install requests


In [3]:
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
# ページデータ取得

import requests
import json

url = 'https://shopping.yahoo.co.jp/categoryranking/42794/list?sc_i=shp_pc_catelist_rankingTab'

page = requests.get(url)

filename =  'page.txt'
with open(filename, mode='w') as f:
    f.write(page.text)

In [16]:
!pip install openpyxl

In [ ]:
# 入力サイトの上位10位データ取得とExcel登録

import requests
from bs4 import BeautifulSoup
from pathlib import Path
import openpyxl

url = input('Yahoo!Shoppingの商品ランキングページURLを貼り付けて下さい')
if url == '':
    url = 'https://shopping.yahoo.co.jp/categoryranking/42794/list?sc_i=shp_pc_catelist_rankingTab'
    
page = BeautifulSoup(requests.get(url).content, 'html.parser')

categories = page.find_all('a', class_ = 'Breadcrumb__link')
l_cat = []
for category in categories:
    l_cat.append(category.text)

# print(l_cat)
# print()

ranks = page.find_all('span', class_ = 'rank-text')
reviews = page.find_all('span', class_ = 'Review__count____2_0_88 Review__count--hasBrackets____2_0_88 review-count')
averages = page.find_all('span', class_ = 'Review__average____2_0_88 review-average')
prices = page.find_all('span', class_ = 'price-number')
names = page.find_all('span', class_ = 'name-text')
images = page.find_all('img', class_ = 'image')

# for (rank, review, average, price, name, image) in zip(ranks, reviews, averages, prices, names, images):
#     print(rank.text, review.text, average.text, price.text, name.text)
#     print(image.get('src'))

# print(len(ranks), len(reviews), len(averages), len(prices), len(names), len(images))

folder = Path('output')
folder.mkdir(exist_ok=True)
excel_path = folder.joinpath('SpreadSheet10.xlsx')

wb = openpyxl.load_workbook(excel_path)
ws = wb['Ranking']

for j in range(len(l_cat)):
    ws.cell(row=6, column=j + 2, value=l_cat[j])

print(len(reviews))
for i in range(len(ranks)):
    ws.cell(row=i + 9, column=3, value=reviews[i].text)
    ws.cell(row=i + 9, column=4, value=averages[i].text)
    ws.cell(row=i + 9, column=5, value=prices[i].text)
    ws.cell(row=i + 9, column=6, value=names[i].text)
    img_url = images[i].get('src')
    ws.cell(row=i + 9, column=7, value=img_url)
    img_data = requests.get(img_url)
    img_path = folder.joinpath(img_url.split('/')[-1] + '.jpg')
    with open(img_path, mode='wb') as f:
        f.write(img_data.content)

wb.save(excel_path)
wb.close()

In [6]:
!pip install pandas

In [ ]:
# pandas経由でcategoryとRankingシートを作成

import pandas as pd

excel_path = folder.joinpath('category.xlsx')
sr = pd.Series(l_cat, name='カテゴリー')

with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    sr.to_excel(writer, sheet_name='category', index=False)

df = pd.DataFrame()
for (rank, review, average, price, name, image) in zip(ranks, reviews, averages, prices, names, images):
    row = {'順位':rank.text, 'レビュー数':review.text, 'アベレージ':average.text, '単価':price.text, '商品名':name.text, '画像':image.get('src')}
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=False)
    df_i = df.set_index('順位')

with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
    df_i.to_excel(writer, sheet_name='ranking', index=True)

In [8]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 11.3 MB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 10.8 MB/s eta 0:00:00a 0:00:01


In [ ]:
# Googleで'Selenium'を検索し、「Seleniumブラウザー自動化プロジェクト」をクリック

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=/Users/ynurmj5e/Library/Application Support/Google/Chrome/Default')
driver = webdriver.Chrome(options=options)
# Googleの検索TOP画面を開く。
driver.get("https://www.google.co.jp/")
# 検索語として「selenium」と入力し、Enterキーを押す。
driver.find_element(By.NAME, "q").send_keys("selenium")
driver.find_element(By.NAME, "q").send_keys(Keys.ENTER)
# タイトルに「Seleniumブラウザー自動化プロジェクト」をクリックする。
driver.find_element(By.CLASS_NAME, "VuuXrf").click()
# 5秒間待機する。
sleep(5)
# ブラウザを終了する。
driver.close()

In [ ]:
# ローカルサーバーテストサイトの「もっと見る」を２度クリック→「閉じる」のテキストをprintしてクリック

import requests
from bs4 import BeautifulSoup
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=/Users/ynurmj5e/Library/Application Support/Google/Chrome/Default')
driver = webdriver.Chrome(options=options)

driver.get("http://10.211.55.7")
sleep(1)

driver.find_element(By.CLASS_NAME, "js-btn-more").click()
sleep(1)

driver.find_element(By.CLASS_NAME, "js-btn-more").click()
sleep(1)

btn = driver.find_element(By.CLASS_NAME, "js-btn-close")
print(btn.text)
btn.click()
sleep(1)

driver.close()


In [ ]:
# Excel記載サイトの『もっと見る」をクリックし、上位30位データを表示・取得してExcel登録

import requests
from bs4 import BeautifulSoup
from pathlib import Path
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

folder = Path('output')
folder.mkdir(exist_ok=True)

excel_path = folder.joinpath('SpreadSheet30.xlsx')
wb = openpyxl.load_workbook(excel_path)
ws = wb['Ranking']
url = ws['C2'].value

page = BeautifulSoup(requests.get(url).content, 'html.parser')
categories = page.find_all('li', class_ = 'Breadcrumb__item')

l_cat = []
for category in categories:
    l_cat.append(category.text)

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--user-data-dir=/Users/ynurmj5e/Library/Application Support/Google/Chrome/Default')
driver = webdriver.Chrome(options=options)

driver.get(url)
driver.maximize_window()
sleep(1)
driver.execute_script("window.scrollTo(0, 2000);")
sleep(1)
more_btn = driver.find_element(By.CLASS_NAME, "button-text").click()
sleep(1)

page = BeautifulSoup(driver.page_source, 'html.parser')

driver.close()

# ranks = page.find_all('span', class_ = 'rank-text')
reviews = page.find_all('span', class_ = 'Review__count____2_0_88 Review__count--hasBrackets____2_0_88 review-count')
averages = page.find_all('span', class_ = 'Review__average____2_0_88 review-average')
prices = page.find_all('span', class_ = 'price-number')
names = page.find_all('span', class_ = 'name-text')
images = page.find_all('img', class_ = 'image')

# for (rank, review, average, price, name, image) in zip(ranks, reviews, averages, prices, names, images):
#     print(rank.text, review.text, average.text, price.text, name.text)
#     print(image.get('src'))

for j in range(len(l_cat)):
    ws.cell(row=6, column=j + 2, value=l_cat[j])

for i in range(len(reviews)):
    ws.cell(row=i + 9, column=3, value=reviews[i].text)
    ws.cell(row=i + 9, column=4, value=averages[i].text)
    ws.cell(row=i + 9, column=5, value=prices[i].text)
    ws.cell(row=i + 9, column=6, value=names[i].text)
    img_url = images[i].get('src')
    ws.cell(row=i + 9, column=7, value=img_url)
    img_data = requests.get(img_url)
    img_path = folder.joinpath(img_url.split('/')[-1] + '.jpg')
    
    with open(img_path, mode='wb') as f:
        f.write(img_data.content)

wb.save(excel_path)
wb.close()


In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium

  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl (118 kB)
Using cached idna-3.6-py3-none-any.whl (61 kB)
Using cached urllib3-2.2.0-py3-none-any.whl (120 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [3]:
# YAHOO!ショッピングのカテゴリーID指定により、その配下のカテゴリーについて、
# 上位10位ランキングデータを取得してExcelに登録する。
# （ページの構成により、レビュー数／アベレージがズレるケース有り）
# Selenium利用により上位30位に拡張可（要コード調整）

import requests
from bs4 import BeautifulSoup
from pathlib import Path
# import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep

from api import ys_api
from exl import exl_wb

SEL_USE = True

# 入力されたカテゴリーIDに従属するID全てをリスト化
ys = ys_api()

while True:
    try:
        id = input('カテゴリーID')
        break
    except ValueError:
        pass
ids = [id]
ys.out_ids = []
ys.list_get_ids(ids)
codes = ys.out_ids

del ys

if SEL_USE:
    # Selenium起動準備
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    options.add_argument('--window-size=3200,1800') 
    options.add_argument('--user-data-dir=/Users/ynurmj5e/Library/Application Support/Google/Chrome/Default')
    driver = webdriver.Chrome(options=options)

# Excelファイル出力準備
wb = exl_wb('output', 'SpreadSheet.xlsx')
if SEL_USE:
    wb.open_workbook(30)
else:
    wb.open_workbook()

# メイン処理
for code in codes:
    # ランキングページ取得
    # url = f'https://shopping.yahoo.co.jp/categoryranking/{code}/list?sc_i=shp_pc_cateranking_nrwcgt'
    url = f'https://shopping.yahoo.co.jp/categoryranking/{code}/list?sc_i=shopping-pc-web-list-ranking-nrwcgt-slctp_md-ranking'
    
    if SEL_USE:
        # Selenium操作
        try:
            driver.get(url)
            sleep(1)
            driver.execute_script("window.scrollTo(0, 2500);")      # 「もっと見る」ボタンが表示されるまでスクロール
            sleep(1)
            driver.execute_script('arguments[0].click();', driver.find_element(By.CLASS_NAME, "button-text"))
            sleep(1)
        except NoSuchElementException:                              # 「もっと見る」ボタンの無いページスキップ
            pass

        page = BeautifulSoup(driver.page_source, 'html.parser')
    else:
        page = BeautifulSoup(requests.get(url).content, 'html.parser')

    # パンくずリスト取得
    categories = page.find_all('li', class_ = 'Breadcrumb__item')
    l_cat = []
    for category in categories:
        l_cat.append(category.text)

    # ランキング情報取得
    ranks = page.find_all('span', class_ = 'rank-text')
    reviews = page.find_all('span', class_ = 'Review__count____2_0_101 Review__count--hasBrackets____2_0_101 review-count')
    averages = page.find_all('span', class_ = 'Review__average____2_0_101 review-average')
    prices = page.find_all('span', class_ = 'price-number')
    names = page.find_all('span', class_ = 'name-text')
    images = page.find_all('img', class_ = 'image')

    # Excel操作
    ws = wb.copy_ws()
    ws.title = code
    ws['C2'].hyperlink = url

    # パンくずリスト登録
    for j in range(len(l_cat)):
        ws.cell(row=6, column=j + 2, value=l_cat[j])

    # ランキング情報登録
    for i in range(len(reviews)):
        ws.cell(row=i + 9, column=3, value=reviews[i].text)
        ws.cell(row=i + 9, column=4, value=float(averages[i].text))
        ws.cell(row=i + 9, column=5, value=int(prices[i].text.replace(',', '')))
        ws.cell(row=i + 9, column=6, value=names[i].text)
        img_url = images[i].get('src')
        ws.cell(row=i + 9, column=7).hyperlink = img_url
        # ws.cell(row=i + 9, column=8).value = f'=_xlfn.image(G{i + 9})'

# Excel終了
wb.save_workbook()

if SEL_USE:
    # Seleniumu終了
    driver.close()
    